## Books!
### Working with different file formats and APIs

- In this task you will ingest data from different sources (XML, JSON, API)
- You will combine the data into one table and write this to a single CSV
- This requires you to handle different file formats and wrangle them to an appropriate format that can be represented as a table

#### ETL Structure
- Create a folder structure to structure your files
- `Raw` - The raw JSON and XML file
- `Bronze` - Your files represented as CSV files
- `Silver` - A single CSV where you've joined all data

In [154]:
## Imports
import json
import csv

## Task 1 - Ingest JSON of Books 
- Read the JSON file to get details of 30 books
- Perform the appropriate wrangling and store it as a CSV in your `Bronze` folder

In [155]:
## Code
json_file_path = "Raw/books_multi.json"
csv_file_out = "Bronze/books.csv"
# Initialize an empty list to store the parsed JSON objects
data = []

with open(json_file_path, 'r') as json_file:
    # Read the lines from the file
    lines = json_file.readlines()

    # Iterate through each line and load the JSON data
    for line in lines:
        # Load each line as a separate JSON object
        json_data = json.loads(line)
        
        # Append the loaded JSON object to the list
        data.append(json_data)

with open(csv_file_out, 'w',newline='') as csv_file:
# Write header    
    # Create a CSV writer
    field_names = list(data[0].keys())
    csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)

    # Write the header to the CSV file
    csv_writer.writeheader()

    # Write the data to the CSV file
    csv_writer.writerows(data)

## Ingest XML of Authors: 
- Parse the XML file for information on the authors
- Perform the appropriate wrangling and store it as a CSV in your `Bronze` folder

In [156]:
## Code needs to be modified to be in a better format for CSV
from lxml import etree
xml_in_path = "Raw/authors.xml"
xml_out_path = 'Bronze/authors.csv'
book_and_author_list = []
books = []
with open(xml_in_path, 'r', newline='') as xml_file:
    # Parse the XML content
    tree = etree.parse(xml_file)
    root = tree.getroot()
    # Now you can work with the XML data using the 'root' element
    # For example, you can iterate through elements
    pog_dict = {}
    for author in root.findall('author'):
        author_name = author.find('name').text

        # Assuming the tag for books is 'books', and the tag for book ISBN is 'book_isbn'
        # books = [book.text for book in author.find('books').findall('book_isbn')]
        
        # Add to the dictionary if the author has books
        # if books:
        #     pog_dict[author_name] = books

        # Write each book and author pair as two elements in a list for simpler CSV creation
        for book in author.find('books').findall('book_isbn'):
            book_and_author_list.append([author_name, book.text])
            books.append(book.text)

print(book_and_author_list)
    # print(pog_dict)
    # print(list(pog_dict.keys()))

[['John Doe', '1000000001'], ['John Doe', '1000000014'], ['John Doe', '1000000027'], ['Jane Smith', '1000000002'], ['Jane Smith', '1000000015'], ['Emily Johnson', '1000000003'], ['Michael Brown', '1000000004'], ['Michael Brown', '1000000016'], ['Sarah Davis', '1000000005'], ['William Miller', '1000000006'], ['William Miller', '1000000017'], ['William Miller', '1000000028'], ['Elizabeth Wilson', '1000000007'], ['Elizabeth Wilson', '1000000018'], ['David Moore', '1000000008'], ['Jennifer Taylor', '1000000009'], ['Jennifer Taylor', '1000000019'], ['Jennifer Taylor', '1000000029'], ['Richard Anderson', '1000000010'], ['Patricia Jackson', '1000000011'], ['Patricia Jackson', '1000000020'], ['Christopher Harris', '1000000012'], ['Christopher Harris', '1000000021'], ['Christopher Harris', '1000000030']]


In [157]:
with open(xml_out_path, 'w', newline='') as xml_file_out:
    field_names = list(pog_dict.keys())
    csv_writer = csv.writer(xml_file_out)

    # Write the header to the CSV file
    header = ['author', 'isbn']
    
    csv_writer.writerow(header)

    # Write the data to the CSV file
    csv_writer.writerows(book_and_author_list)

In [158]:
# OLD CODE FOR DICTIONARY SETUP
# with open(xml_out_path, 'w', newline='') as xml_file_out:
#     field_names = list(pog_dict.keys())
#     csv_writer = csv.DictWriter(xml_file_out, fieldnames=field_names)

#     # Write the header to the CSV file
#     csv_writer.writeheader()

#     # Write the data to the CSV file
#     csv_writer.writerow(pog_dict)

## Fetch Ratings from API: 
- For each book, use the Random Numbers API to fetch 10 ratings (1 to 5)
- The endpoint you'll use is: "http://www.randomnumberapi.com/api/v1.0/random?min=1&max=5&count=10"
- Store the ratings in a CSV in your `Bronze` folder

In [159]:
import requests

In [160]:
# flatten out book_list to a pure list of books with no duplicates
# this is where book_list is a combined list with authors and books  , no need to do this when we have a books list with only the book ids
# flattened_list = []
# seen_books = set()

# for sublist in book_list:
#     for book in sublist:
#         if book not in seen_books:
#             flattened_list.append(book)
#             seen_books.add(book)

# # Print the result
# print(flattened_list)

In [161]:
## Code
min = 1
max = 5
count = 10
def get_ratings():
    response = requests.get(f"https://www.randomnumberapi.com/api/v1.0/random?min={min}&max={max}&count={count}")

    if not response.ok:
        print("Request was bad!")
        return False
    # If we use the response.json() function, we get a dict back!
    type(response.json())

    json_response = response.json()
    return json_response

In [162]:
#Get the ratings

# print(get_ratings())
book_ratings = {}
for book in books:
    book_ratings[book] = get_ratings()



In [163]:
print(book_ratings)
book_ratings_path = 'Bronze/book_ratings.csv'
with open(book_ratings_path, 'w', newline='') as book_csv_out:

    csv_writer = csv.writer(book_csv_out)

    # Write the header to the CSV file
    csv_writer.writerow(["book_isbn", "rating_list"])

    # Write the data to the CSV file
    for isbn, ratings in book_ratings.items():
        csv_writer.writerow([isbn, ratings])


{'1000000001': [4, 4, 3, 3, 2, 1, 4, 2, 2, 2], '1000000014': [3, 2, 1, 3, 3, 3, 1, 2, 1, 1], '1000000027': [4, 3, 4, 4, 3, 4, 1, 2, 2, 3], '1000000002': [1, 1, 1, 3, 3, 4, 4, 1, 1, 4], '1000000015': [4, 2, 1, 2, 3, 3, 3, 2, 2, 4], '1000000003': [3, 1, 3, 4, 2, 2, 3, 2, 4, 2], '1000000004': [3, 4, 4, 1, 1, 2, 3, 1, 2, 2], '1000000016': [1, 1, 3, 4, 1, 1, 4, 4, 1, 2], '1000000005': [2, 3, 4, 4, 3, 1, 3, 3, 2, 4], '1000000006': [2, 3, 1, 2, 1, 3, 1, 1, 2, 3], '1000000017': [2, 4, 4, 1, 1, 2, 1, 4, 3, 1], '1000000028': [4, 4, 2, 2, 2, 2, 1, 3, 2, 3], '1000000007': [4, 4, 1, 1, 2, 2, 1, 1, 4, 1], '1000000018': [1, 3, 3, 2, 2, 2, 3, 3, 3, 3], '1000000008': [2, 1, 1, 4, 2, 3, 2, 4, 2, 3], '1000000009': [3, 3, 2, 3, 2, 3, 1, 2, 3, 1], '1000000019': [2, 1, 1, 4, 1, 4, 3, 1, 3, 1], '1000000029': [3, 2, 2, 4, 3, 4, 4, 3, 4, 3], '1000000010': [3, 2, 2, 4, 3, 2, 2, 4, 2, 4], '1000000011': [2, 1, 4, 3, 3, 4, 1, 1, 4, 4], '1000000020': [1, 4, 3, 4, 4, 3, 2, 4, 2, 3], '1000000012': [1, 1, 2, 3, 2, 1, 

In [164]:
# Books.csv each books ISBN has a title and publication year tied to them
# isbn,title,publication_year
# 1000000001,Data Structures in Python,2020

# Rating.csv, each ISBN number has a list of ratings tied to them
# book_isbn,rating_list
# 1000000001,"[4, 1, 4, 2, 1, 3, 4, 3, 1, 2]"

# Authors.csv, each Author has a list of ISBN for books tied to them
# John Doe,Jane Smith,Emily 
# "['1000000001', '1000000014', '1000000027']","['1000000002', '1000000015']"


# books_Complete = {}
# books_complete[books.isbn] = [books.title, books.publication_year, authors.author, rating.ratinglist]



# Dictionary to store the combined information
books_complete = {}

# Paths to your CSV files
books_path = 'Bronze/books.csv'
rating_path = 'Bronze/book_ratings.csv'
authors_path = 'Bronze/authors.csv'

# Read data from Books.csv
with open(books_path, 'r') as books_file:
    books_reader = csv.DictReader(books_file)
    for row in books_reader:
        isbn = row['isbn']
        title = row['title']
        publication_year = row['publication_year']
        books_complete[isbn] = [title, publication_year]

print(books_complete)

# # Read data from Authors.csv
# with open(authors_path, 'r') as authors_file:
#     authors_reader = csv.reader(authors_file)
#     next(authors_reader)  # Skip the header
#     for row in authors_reader:
#         author_name = row[0]
#         isbn_list = ast.literal_eval(row[1])  # Safely convert the string to a list
#         for isbn in isbn_list:
#             if isbn in books_complete:
#                 books_complete[isbn].append(author_name)


# # Read data from Rating.csv
# with open(rating_path, 'r') as rating_file:
#     rating_reader = csv.DictReader(rating_file)
#     for row in rating_reader:
#         isbn = row['book_isbn']
#         rating_list = ast.literal_eval(row['rating_list'])  # Safely convert the string to a list
#         if isbn in books_complete:
#             books_complete[isbn].append(rating_list)

# # Print the combined information
# for isbn, info in books_complete.items():
#     print(f"ISBN: {isbn}, Title: {info[0]}, Publication Year: {info[1]}, Author: {info[2]}, Ratings: {info[3]}")



{'1000000001': ['Data Structures in Python', '2020'], '1000000002': ['Introduction to ETL Processes', '2019'], '1000000003': ['APIs for Beginners', '2021'], '1000000004': ['Machine Learning Fundamentals', '2018'], '1000000005': ['Database Management Essentials', '2022'], '1000000006': ['Cloud Computing Basics', '2020'], '1000000007': ['The Art of Data Science', '2019'], '1000000008': ['Python for Data Analysis', '2021'], '1000000009': ['Algorithms Unlocked', '2020'], '1000000010': ['Web Development with JavaScript', '2018'], '1000000011': ['Introduction to IoT', '2022'], '1000000012': ['Networking for Beginners', '2019'], '1000000013': ['Cybersecurity Fundamentals', '2020'], '1000000014': ['Big Data Analysis', '2021'], '1000000015': ['Programming in Java', '2018'], '1000000016': ['HTML and CSS Design', '2022'], '1000000017': ['Software Testing Techniques', '2019'], '1000000018': ['Blockchain Basics', '2020'], '1000000019': ['Artificial Intelligence Concepts', '2021'], '1000000020': ['Q

In [165]:
# # Read data from Authors.csv
# import ast
# with open(authors_path, 'r') as authors_file:
#     authors_reader = csv.reader(authors_file)
#     # next(authors_reader)  # Skip the header
#     for row in authors_reader:
#         print(row[0])
#         author_name = row[0]
#         # isbn_list = ast.literal_eval(row[1])  # Safely convert the string to a list
#         # for isbn in isbn_list:
#         #     if isbn in books_complete:
#         #         books_complete[isbn].append(author_name)

## Combine Data 
- Create a comprehensive dataset where each book entry includes its title, publication year, author, and its average rating.
- Store this as a CSV in your `Silver` folder

In [166]:
authors_csv_path = "Bronze/authors.csv"
book_ratings_csv = "Bronze/book_ratings.csv"
books_csv = "Bronze/books.csv"

csv_list = [authors_csv_path, book_ratings_path, books_csv]



def read_return_csv(path):
    with open(path, 'r', newline='') as file:
        csvit = csv.DictReader(file)
        return_list = []
        for row in csvit:
            return_list.append(row)

        return return_list
        



# with open(authors_csv_path, 'r', newline='') as author_csv_file:
#     csv_authors = csv.reader(author_csv_file)
#     for row in csv_authors:
#         print(row)

# with open(book_ratings_csv, 'r', newline='') as book_ratings_csv_file:
#     csv_book_ratings = csv.reader(book_ratings_csv_file)
#     for row in csv_book_ratings:
#         print(row)

# with open(books_csv, 'r', newline='') as book_csv_file:
#     csv_books = csv.reader(book_csv_file)
#     for row in csv_books:
#         print(row)

In [167]:
complete_list = []

for i in csv_list:
    complete_list.append(read_return_csv(i))


author_isbn = complete_list[0]
book_rating = complete_list[1]
title_pub = complete_list[2]



In [247]:
#LIST METHOD
merged = [["isbn", "title", "publication_year", "rating_list", "author"]]
for author in author_isbn:
    for book in book_rating:
        if author["isbn"] == book["book_isbn"]:
            for title in title_pub:
                if(author["isbn"] == title["isbn"]):
                    # merged[book["book_isbn"]] = [title["title"], title["publication_year"], book["rating_list"], author["author"]]
                    merged.append([book["book_isbn"],title["title"], title["publication_year"], book["rating_list"], author["author"]])
                
print(merged)
# for author in author_isbn:
#     print(author)
#     merged[author[1]] = author[0]

# for i in book_rating:
#     print(i)

# for i in title_pub:
#     print(i)

# print(merged)

# for lists in complete_list:
#     for i in lists:
#         print(i[1])

[['isbn', 'title', 'publication_year', 'rating_list', 'author'], ['1000000001', 'Data Structures in Python', '2020', '[4, 4, 3, 3, 2, 1, 4, 2, 2, 2]', 'John Doe'], ['1000000014', 'Big Data Analysis', '2021', '[3, 2, 1, 3, 3, 3, 1, 2, 1, 1]', 'John Doe'], ['1000000027', 'Mobile App Design Essentials', '2019', '[4, 3, 4, 4, 3, 4, 1, 2, 2, 3]', 'John Doe'], ['1000000002', 'Introduction to ETL Processes', '2019', '[1, 1, 1, 3, 3, 4, 4, 1, 1, 4]', 'Jane Smith'], ['1000000015', 'Programming in Java', '2018', '[4, 2, 1, 2, 3, 3, 3, 2, 2, 4]', 'Jane Smith'], ['1000000003', 'APIs for Beginners', '2021', '[3, 1, 3, 4, 2, 2, 3, 2, 4, 2]', 'Emily Johnson'], ['1000000004', 'Machine Learning Fundamentals', '2018', '[3, 4, 4, 1, 1, 2, 3, 1, 2, 2]', 'Michael Brown'], ['1000000016', 'HTML and CSS Design', '2022', '[1, 1, 3, 4, 1, 1, 4, 4, 1, 2]', 'Michael Brown'], ['1000000005', 'Database Management Essentials', '2022', '[2, 3, 4, 4, 3, 1, 3, 3, 2, 4]', 'Sarah Davis'], ['1000000006', 'Cloud Computing B

In [250]:
# LIST METHOD
merged_path = 'Silver/merged.csv'
with open(merged_path, 'w', newline='') as csv_file_out:
    field_names = ["isbn", "Info"]
    csv_writer = csv.writer(csv_file_out)

    # Write the header to the CSV file
    # csv_writer.writerow(merged[0])

    # # Write the rest of data to the CSV file
    # csv_writer.writerows(merged[1:])
    csv_writer.writerows(merged)

In [ ]:
#DICT METHOD
merged = [["isbn", "title", "publication_year", "rating_list", "author"]]
for author in author_isbn:
    for book in book_rating:
        if author["isbn"] == book["book_isbn"]:
            for title in title_pub:
                if(author["isbn"] == title["isbn"]):
                    # merged[book["book_isbn"]] = [title["title"], title["publication_year"], book["rating_list"], author["author"]]
                    merged.append([book["book_isbn"],title["title"], title["publication_year"], book["rating_list"], author["author"]])
                
print(merged)